# How to implement a if-based constraint in MILP

We want to expand the MILP defined in [`limit energy notebook`](limit-energy.ipynb) by adding the following constraint:


$$
E_t^{lower} =
\begin{cases}
E_{t-1} + Eh_t, & \text{if } E_{t-1} \le 0 \\
E_{t-1} + Eh_t - Ec_t, & \text{if } E_{t-1} > 0
\end{cases}
$$

where $E_t$ is the energy level at time $t$, $Eh_t$ is the energy harvested at time $t$, and $Ec_t$ is the energy consumed at time $t$.

At the same time, we want to limit the energy level to a maximum value $E_{\max}$.

$$
E_t = \min\big(E_{\max}, \; E_t^{lower}\big), \forall t \in \{1,\dots,T\}
$$

with $E_0 = \text{const}$.

This is a conditional update, which is not linear as written, but we still can encode it in MILP using a binary variable and the big-M trick.


### MILP Formulation

* Variables:

  * $E_t \in [0, E_{\max}]$ (continuous, energy at time $t$)
  * $y_t \in \{0,1\}$ (binary, indicates whether the cap $E_{\max}$ is binding at time $t$)
  * $z_t \in \{0,1\}$ (binary, indicates whether $E_{t-1} > 0$ as shown below)

$$
z_t =
\begin{cases}
1 & \text{if } E_{t-1} > 0 \\
0 & \text{if } E_{t-1} \le 0
\end{cases}
$$


* Parameters:

  * $E_{\max}$ (capacity, constant)
  * $Eh_t = f(t)$   (energy harvested at time $t$)
  * $Ec_t = g(t)$   (energy consumed at time $t$)


#### Constraints

For all $t = 1,\dots,T$:

1. Initial condition

$$
E_0 = \text{const}.
$$

2. The unified equation is:

$$
E_t^{lower} = E_{t-1} + Eh_t - z_t\cdot Ec_t
$$

3. Enforcing consistency of first equation with $z_t$:

We need constraints to tie $z_t$ with the condition on $E_{t-1}$:

* If $E_{t-1} \le 0$, then $z_t=0$.
* If $E_{t-1} > 0$, then $z_t=1$.

Those conditions can be expressed using **big-M constraints** with a sufficiently large $M$:

$$
E_{t-1} \le M \cdot z_t
$$

$$
E_{t-1} \ge \epsilon - M \cdot (1 - z_t)
$$

Here $\epsilon > 0$ is a small constant to break ties (e.g. $10^{-6}$).


4. Limit maximum - upper bounds

$$
E_t \le E_t^{lower}
$$

$$
E_t \le E_{\max}
$$

5. Limit maximum - Big-M linking (with $M = E_{\max}$)

$$
E_t \ge E_t^{lower} - E_{\max} \cdot y_t
$$

$$
E_t \ge E_{\max} - E_{\max}\cdot(1-y_t)
$$


In [156]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
import random
import pyomo.environ as pyo
import matplotlib.pyplot as plt

In [158]:
model = pyo.ConcreteModel()

Tmax = 200 # maximum time

Emax = 28  # maximum energy
E0_const = 5  # initial energy

In [159]:
# model variables

model.T = pyo.RangeSet(1, Tmax)
model.T_all = pyo.RangeSet(0, Tmax)   # includes 0

model.E = pyo.Var(model.T_all, within=pyo.Reals, bounds=(-Emax, Emax))
model.y = pyo.Var(model.T, within=pyo.Binary)

model.z = pyo.Var(model.T, within=pyo.Binary)

In [160]:
# Initial condition
model.E[0].fix(E0_const)

In [161]:
Eh = [random.randint(1, 5)  for t in model.T]  # energy harvested at time t
Ec = [random.randint(1, 5)  for t in model.T]  # energy consumed at time t


In [162]:
# for the system to be feasible with the values below, Emax must be at least 28 with E0 = 5

Eh = [5, 5, 1, 2, 5, 3, 1, 1, 5, 4, 1, 5, 3, 4, 5, 4, 4, 4, 4, 4, 4, 3, 2, 1, 4, 5, 1, 2, 2, 2, 1, 3, 1, 4, 2, 2, 4, 5, 5, 2, 3, 3, 4, 4, 5, 1, 2, 3, 1, 2, 1, 2, 4, 2, 3, 1, 1, 4, 3, 1, 3, 1, 2, 3, 3, 5, 2, 1, 3, 2, 5, 3, 5, 2, 3, 5, 3, 3, 4, 3, 4, 4, 4, 5, 1, 3, 2, 1, 5, 3, 1, 1, 4, 5, 4, 2, 2, 1, 2, 5, 5, 2, 2, 4, 5, 5, 5, 5, 3, 2, 5, 1, 1, 4, 1, 4, 1, 1, 1, 5, 3, 4, 2, 3, 5, 2, 5, 4, 3, 3, 5, 4, 1, 1, 1, 1, 4, 2, 4, 2, 2, 3, 4, 2, 2, 5, 4, 2, 3, 4, 1, 2, 4, 2, 1, 4, 4, 2, 1, 4, 4, 1, 1, 1, 4, 2, 4, 5, 3, 4, 4, 1, 4, 1, 3, 4, 3, 5, 3, 4, 4, 2, 4, 4, 1, 1, 2, 5, 5, 1, 4, 5, 2, 1, 3, 4, 3, 3, 2, 2]
Ec = [1, 3, 1, 4, 2, 1, 1, 1, 3, 4, 4, 5, 3, 3, 4, 2, 5, 3, 4, 3, 1, 5, 2, 1, 2, 2, 2, 3, 5, 4, 4, 5, 3, 1, 3, 3, 2, 2, 4, 1, 5, 3, 1, 4, 3, 4, 3, 4, 5, 4, 2, 5, 3, 1, 3, 3, 5, 1, 5, 2, 4, 2, 1, 2, 1, 1, 1, 4, 2, 2, 5, 1, 3, 1, 3, 4, 4, 3, 3, 4, 2, 2, 1, 1, 1, 4, 2, 1, 5, 4, 2, 4, 5, 1, 1, 3, 3, 1, 3, 2, 1, 2, 5, 5, 3, 2, 3, 1, 3, 2, 5, 1, 2, 4, 3, 2, 5, 2, 3, 3, 4, 5, 3, 2, 2, 3, 3, 4, 4, 5, 5, 1, 3, 2, 3, 3, 4, 1, 1, 3, 2, 3, 5, 4, 3, 3, 3, 4, 3, 5, 5, 1, 4, 2, 1, 4, 2, 2, 2, 1, 4, 1, 5, 4, 5, 3, 4, 2, 3, 3, 5, 4, 2, 4, 4, 5, 1, 2, 4, 3, 5, 3, 4, 2, 5, 5, 4, 3, 5, 3, 3, 1, 2, 4, 2, 4, 1, 5, 2, 2]

# Model is not covnerging after several hours of running

In [ ]:
M = 1000 * Emax  # Big-M value
epsilon = 1e-6  # 1e-10

model.EnergyConstr = pyo.ConstraintList()
for t, Eh_t, Ec_t in zip(model.T, Eh, Ec):


    # Big-M constraints for z[t] indicating E[t] > 0
    model.EnergyConstr.add(model.E[t-1] <= M * model.z[t])
    model.EnergyConstr.add(model.E[t-1] >= epsilon - M * (1 - model.z[t]))

    # Upper bounds
    model.EnergyConstr.add(model.E[t] <= model.E[t-1] + Eh_t - model.z[t] * Ec_t)
    model.EnergyConstr.add(model.E[t] <= Emax)

    # Max. Capacity binding constraints using y[t]
    model.EnergyConstr.add(model.E[t] >= model.E[t-1] + Eh_t - model.z[t] * Ec_t - Emax * model.y[t])
    model.EnergyConstr.add(model.E[t] <= Emax - Emax * (1 - model.y[t]))


In [164]:
# Objective: maximize sum of E(t)
model.Obj = pyo.Objective(expr=sum(model.E[t] for t in model.T),
                          sense=pyo.maximize)

In [ ]:
# Solver (make sure you have glpk or cbc installed)
solver = pyo.SolverFactory("glpk")

# Solve
results = solver.solve(model, tee=True)
# Check solver status
print(results.solver.status)
print(results.solver.termination_condition)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpes1qf5r1.glpk.raw --wglp /tmp/tmpm1kq7ehx.glpk.glp --cpxlp
 /tmp/tmp81xuc85h.pyomo.lp
Reading problem data from '/tmp/tmp81xuc85h.pyomo.lp'...
/tmp/tmp81xuc85h.pyomo.lp:7206: warning: lower bound of variable 'x203' redefined
/tmp/tmp81xuc85h.pyomo.lp:7206: warning: upper bound of variable 'x203' redefined
1200 rows, 600 columns, 2796 non-zeros
400 integer variables, all of which are binary
7606 lines were read
Writing problem data to '/tmp/tmpm1kq7ehx.glpk.glp'...
6199 lines were written
GLPK Integer Optimizer 5.0
1200 rows, 600 columns, 2796 non-zeros
400 integer variables, all of which are binary
Preprocessing...
402 constraint coefficient(s) were reduced
997 rows, 599 columns, 2591 non-zeros
399 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  2.800e+01  ratio =  2.800e+01
GM: min|aij| =  1.890e-01  max|aij| =  5.292e+00  ratio =  2.800e+01
EQ: min|

In [ ]:
E_vals = [pyo.value(model.E[t]) for t in model.T_all]
ts = list(range(Tmax + 1))

plt.figure(figsize=(8, 5))
plt.plot(ts, E_vals, label="Stored Energy")
plt.plot(ts[1:], Eh, linestyle="-", label="Harvested Energy")
plt.plot(ts[1:], Ec, linestyle="-", label="Consumed Energy")
plt.xlabel("Time step t")
plt.ylabel("Value")
plt.title("Energy Storage and Harvested Energy over Time")
plt.legend()
plt.grid(True)
plt.show()


ERROR: evaluating object as numeric value: E[1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized VarData object E[1]


ValueError: No value for uninitialized VarData object E[1]

Look between t = 100 and t = 125 and you will see that the energy is capped at E_max = 30.

> Note that this is a simplified example and we are not controlling the consumed energy. In a real-world scenario, you you cannot consume energy if E_t = 0. So if you change Emax to a lower value, the system can become infeasible.